<a href="https://colab.research.google.com/github/EngenhariaSoftwarePUCRS/Inteligencia_Artificial/blob/develop/Trabalho01/Trabalho01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
integrantes = ["Augusto Baldino", "Felipe Freitas", "Isabela Kuser", "Luiza Heller", "Maria Eduarda Maia", "Paola Lopes"]
useful_links = [
    "https://pandas.pydata.org/pandas-docs/version/1.0/user_guide/style.html",
    "https://www.geeksforgeeks.org/how-to-replace-values-in-column-based-on-condition-in-pandas/",
    "https://www.geeksforgeeks.org/how-to-add-header-row-to-a-pandas-dataframe/",
    "https://discuss.streamlit.io/t/is-it-possible-to-center-data-on-cell-like-excel/32045/7",
]

In [2]:
# This block contains all required imports
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.spatial.distance import cdist
from sklearn import (
    cluster,
    metrics,
    model_selection,
    neighbors,
    neural_network,
    tree,
)
from typing import Literal

In [3]:
# Wheter to print or not
DEBUG = True
old_print = print
if old_print:
    print = old_print
if DEBUG == False:
    def do_nothing(*args, **kwargs):
        pass
    print = do_nothing

In [4]:
# This block contains the dataset initial import and column names
targetClass = "Outcome"
columnNames = [
    "Top-Left", "Top-Middle", "Top-Right",
    "Middle-Left", "Middle-Middle", "Middle-Right",
    "Bottom-Left", "Bottom-Middle", "Bottom-Right",
    targetClass,
]
dataset = pd.read_csv(
    "data/tic-tac-toe-full.data",
    names=columnNames,
)

In [5]:
# This block contains possible game output values
X_GANHOU = "X Ganhou"
O_GANHOU = "O Ganhou"
VELHA = "Deu Velha"
TEM_JOGO = "Tem Jogo"
gameOutput = Literal["X Ganhou", "O Ganhou", "Deu Velha", "Tem Jogo"]

In [6]:
# This block handles cell logic and field values
# It is used for guaranteeing consistency when comparing diferent cells
cellStrType = Literal['x', 'b', 'o']
cellNumType = Literal[1, 0, -1]
cellEntry = cellStrType | cellNumType

class Cell:
    raw: cellStrType
    value: cellNumType
    def __init__(self, raw: cellStrType, value: cellNumType):
        self.raw = raw
        self.value = value

    def __str__(self) -> str:
        if (self.raw == 'b'):
            return ' '
        return self.raw

    def __repr__(self) -> str:
        return f"<Cell {self.raw} | {self.value}>"

    def get_entries(self) -> list[cellEntry]:
        return [self.raw, self.value]

CELL_X = Cell('x', 1)
CELL_O = Cell('o', -1)
CELL_B = Cell('b', 0)

In [7]:
# This block is a function that converts a cell entry to a valid cell
# It is used to guarantee consistency when handling the data values
def to_cell(cell: cellEntry) -> Cell:
    # Python...
    if isinstance(cell, Cell):
        return cell

    for cell_model in [CELL_X, CELL_B, CELL_O]:
        if cell in cell_model.get_entries():
            return cell_model

    raise Exception(f"Invalid entry '{cell}', please check your dataset")

In [8]:
# This block is a function that prints the board in a human-readable format
# It is used mostly for debugging
def print_board(board: list[cellEntry]):
    for i in range(0, 9, 3):
        row_raw = board[i:i+3]
        row_cell = [to_cell(cell) for cell in row_raw]
        row_string = [str(cell) for cell in row_cell]
        row_pretty = " | ".join(row_string)
        print(row_pretty)

In [9]:
# This block is a function that checks wether a player has won the game
# It is used on the initial dataset transformation, to convert the target column appropriately
def has_won(player: Cell, board: list[Cell]) -> bool:
    if len(board) != 9:
        raise ValueError("Board cannot have any different number of rows x columns than 9")

    for i in range(0, 9, 3):
        line = board[i:i+3]
        # print("Line: ", line)
        if all(cell == player for cell in line):
            return True

    for i in range(0, 3):
        column = [board[i], board[i+3], board[i+6]]
        # print("Column: ", column)
        if all(cell == player for cell in column):
            return True

    if board[4] != player:
        return False

    diagonal1 = [board[0], board[8]]
    # print("D1: ", diagonal1)
    if all(cell == player for cell in diagonal1):
        return True

    diagonal2 = [board[2], board[6]]
    # print("D2: ", diagonal2)
    if all(cell == player for cell in diagonal2):
        return True

    return False

In [10]:
# Simply check wether dataset appears to be correct on its initial state
dataset.head(3)

,Top-Left,Top-Middle,Top-Right,Middle-Left,Middle-Middle,Middle-Right,Bottom-Left,Bottom-Middle,Bottom-Right,Outcome
0,x,x,x,x,o,o,x,o,o,positive
1,x,x,x,x,o,o,o,x,o,positive
2,x,x,x,x,o,o,o,o,x,positive


In [11]:
# This block handles the majority of the dataset's transformation
# It should convert all cells into their numeric counterparts
# It should also adjust the output column to the proper value
for index, row in dataset.iterrows():
    board: list[Cell] = []
    for i, cell_str in enumerate(row.array.tolist()[:-1]):
        columnName = columnNames[i]
        cell = to_cell(cell_str)
        dataset.at[index, columnName] = cell.value
        board.append(cell)

    if (row[targetClass] == 'positive') or has_won(CELL_X, board):
        targetValue = X_GANHOU
    elif (row[targetClass] == 'negative') or has_won(CELL_O, board):
        targetValue = O_GANHOU
    elif (row[targetClass] == 'neutral') or CELL_B in board:
        targetValue = TEM_JOGO
    elif (row[targetClass] == 'tie') or True:
        targetValue = VELHA

    dataset.at[index, targetClass] = targetValue

In [12]:
# Simply check wether dataset appears to be correct on its desired state
dataset.head(3)

,Top-Left,Top-Middle,Top-Right,Middle-Left,Middle-Middle,Middle-Right,Bottom-Left,Bottom-Middle,Bottom-Right,Outcome
0,1,1,1,1,-1,-1,1,-1,-1,X Ganhou
1,1,1,1,1,-1,-1,-1,1,-1,X Ganhou
2,1,1,1,1,-1,-1,-1,-1,1,X Ganhou


In [13]:
# Properly check wether dataset was fully transformed to desired outputs (numerically)
x_ganhou_count = dataset[targetClass].eq(X_GANHOU).sum()
o_ganhou_count = dataset[targetClass].eq(O_GANHOU).sum()
deu_velha_count = dataset[targetClass].eq(VELHA).sum()
tem_jogo_count = dataset[targetClass].eq(TEM_JOGO).sum()

outcomes: dict[gameOutput, int] = {
    X_GANHOU: x_ganhou_count,
    O_GANHOU: o_ganhou_count,
    VELHA: deu_velha_count,
    TEM_JOGO: tem_jogo_count,
}

# Calculate the length of each header for formatting
outcome_length = max(len(outcome) for outcome in outcomes.keys())
count_length = max(len(str(count)) for count in outcomes.values())
count_length = max(count_length, len(" Count "))
percentage_length = len("Percentage")

print(f"| {'Outcome':<{outcome_length}} | {'Count':<{count_length}} | {'Percentage':<{percentage_length}} |")
print(f"|{'-' * (outcome_length + 2)}|{'-' * (count_length + 2)}|{'-' * (percentage_length + 2)}|")
for outcome, count in outcomes.items():
    percentage = (count / len(dataset)) * 100
    print(f"| {outcome:<{outcome_length}} | {count:<{count_length}} | {percentage:>{percentage_length - 1}.4f}% |")

In [14]:
# This block splits the dataset equally for testing across different models

# Define the features (X) and the target variable (y)
X = dataset.drop(columns=[targetClass])
y = dataset[targetClass]

# Define the split ratios
split_ratios = {'train': 0.8, 'validation': 0.05, 'test': 0.15}
# Assure that it adds to 1
if sum(split_ratios.values()) != 1.0:
    raise Exception("Ratios must add up to 1.0")
validation_plus_test_ratio = split_ratios['validation'] + split_ratios['test']
test_to_validation_ratio = split_ratios['test'] / validation_plus_test_ratio

# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = model_selection.train_test_split(
    X, y, train_size=split_ratios['train'], stratify=y, random_state=42)
X_validation, X_test, y_validation, y_test = model_selection.train_test_split(
    X_temp, y_temp, test_size=test_to_validation_ratio, stratify=y_temp, random_state=42)

# Print the shapes of the resulting sets to verify the split
# Print the distribution of outcomes in each set
print("Training set shape/distribution:", X_train.shape)
print(y_train.value_counts(normalize=True))

print("\nValidation set shape/distribution:", X_validation.shape)
print(y_validation.value_counts(normalize=True))

print("\nTest set shape/distribution:", X_test.shape)
print(y_test.value_counts(normalize=True))

In [15]:
# This block uses the validation set to check wether the dataset is correct
for i in range(len(X_validation)):
    print("Linha", i, "Tabuleiro:")
    print_board(X_validation.iloc[i].values)
    print("Outcome: ", y_validation.iloc[i], end="\n\n")

<center><h2>kNN Section</h2></center>

In [16]:
# Baseado no exemplo 1 de kNN; semana 4

if DEBUG == True:
    X = dataset.drop(columns=[targetClass]).values
    X = np.array(X, dtype=np.float64)

    # Distorção: Calcula a distância dos pontos de cada cluster até seu centróide
    distortions = []
    # Inércia: Distância dos dados até o centróide mais próximos
    inertias = []

    # Testando valores de 1 a [16] para o número de clusters
    K = range(1, outcomes.get(VELHA, 0) * 2 + 1)

    for k in K:
        # Executando o K-Means para cada valor de k
        kmeanModel = cluster.KMeans(n_clusters=k, n_init='auto')
        kmeanModel.fit(X)

        # Calcula a distância de cada ponto de X ao centróide
        distancias = cdist(X, kmeanModel.cluster_centers_, 'euclidean')

        soma = 0
        for item in distancias:
            soma += min(item)
        coesao = soma / len(distancias)

        distortions.append(coesao)
        inertias.append(kmeanModel.inertia_)


    # Plotting the distortion
    plt.plot(K, distortions, 'bx-')
    plt.xlabel('Values of K')
    plt.ylabel('Distortion')
    plt.title('Elbow Method using Distortion')
    plt.grid(True)
    plt.show()

    print()

    # Plotting the inertia
    plt.plot(K, inertias, 'bx-')
    plt.xlabel('Values of K')
    plt.ylabel('Inertia')
    plt.title('Elbow Method using Inertia')
    plt.grid(True)
    plt.show()

In [17]:
# This block defines the best k for the kNN algorithm (tests k's accuracy)
# for outcome, count in outcomes.items():
best_k = 16
best_accuracy = 0
for k in range(1, outcomes.get(VELHA, 0) * 2):
    kNN_classifier = neighbors.KNeighborsClassifier(n_neighbors=k)
    kNN_classifier.fit(X_train, y_train)
    predictions_test = kNN_classifier.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, predictions_test)
    if accuracy >= best_accuracy:
        best_k = k
        best_accuracy = accuracy
    print(f"k-NN accuracy ({k}): {accuracy:.4f}")

print("Best: ", best_k, "\nAccuracy: ", best_accuracy)

In [18]:
# Actually setup kNN
kNN = neighbors.KNeighborsClassifier(n_neighbors=best_k)
kNN.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=7)

In [19]:
kNNTestScore = kNN.score(X_test, y_test)
print("Test Accuracy: ", kNNTestScore)
kNNValidationScore = kNN.score(X_validation, y_validation)
print("Validation Accuracy: ", kNNValidationScore)

<center><h2>MLP Section</h2></center>

In [20]:
MLP = neural_network.MLPClassifier(
    random_state=42,
    solver="sgd",
    hidden_layer_sizes=(9, 90, 4),
    learning_rate_init=0.1,
    max_iter=2048,
    verbose=DEBUG,
)
MLP.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(9, 90, 4), learning_rate_init=0.1,
              max_iter=2048, random_state=42, solver='sgd')

In [21]:
MLPTestPredictions = MLP.predict(X_test)
print("Test Confusion Matrix:\n",
      metrics.confusion_matrix(y_test, MLPTestPredictions))
print("Test Classification:\n",
      metrics.classification_report(y_test, MLPTestPredictions, zero_division=0))

In [22]:
MLPValidationPredictions = MLP.predict(X_validation)
print("Validation Confusion Matrix:\n",
      metrics.confusion_matrix(y_validation, MLPValidationPredictions))
print("Validation Classification:\n",
      metrics.classification_report(y_validation, MLPValidationPredictions, zero_division=0))

<center><h2>Decision Tree Section</h2></center>

In [23]:
DTree = tree.DecisionTreeClassifier(
    random_state=42,
    criterion='gini',
    class_weight='balanced',
)

In [24]:
DTree.fit(X_train, y_train)
dtree_predictions = DTree.predict(X_test)

In [25]:
print("Confusion Matrix:\n", metrics.confusion_matrix(y_test, dtree_predictions))
print("Accuracy: ", metrics.accuracy_score(y_test, dtree_predictions))
print(metrics.classification_report(y_test, dtree_predictions, zero_division=0))

<center><h2>Results Section</h2></center>

In [26]:
def get_game_state(board: list[cellEntry], algorithm: Literal["kNN", "MLP", "DTree"] = "kNN") -> gameOutput:
    input_cell = [to_cell(cell) for cell in board]
    input_numeric = [cell.value for cell in input_cell]
    input = {columnNames[i]: input_numeric[i] for i in range(len(input_numeric))}
    input = pd.DataFrame([input])

    if algorithm == "kNN":
        outcome, *_ = kNN.predict(input)
        return outcome

    if algorithm == "MLP":
        outcome, *_ = MLP.predict(input)
        return outcome

    if algorithm == "DTree":
        outcome, *_ = DTree.predict(input)
        return outcome

    raise Exception("Invalid Algorithm")

In [27]:
# This last block can be used as a "playground" to test the models
game_state: list[cellEntry] = [
    'x', 'x', 'o',
    'o', 'x', 'x',
    'x', 'o', 'o',
]

game_state_cells = [to_cell(cell) for cell in game_state]
if has_won(CELL_X, game_state_cells):
    correct_output = X_GANHOU
elif has_won(CELL_O, game_state_cells):
    correct_output = O_GANHOU
elif CELL_B in game_state_cells:
    correct_output = TEM_JOGO
else:
    correct_output = VELHA
print("Saída Correta: ", correct_output)

print("Saída kNN: ", get_game_state(game_state, "kNN"))
print("Saída MLP: ", get_game_state(game_state, "MLP"))
print("Saída DTree: ", get_game_state(game_state, "DTree"))

In [28]:
# Restore print function if needed
if DEBUG == False:
    print = old_print